# Import Libraries 

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import difflib
import re
from tqdm.auto import tqdm
from multiprocessing import Pool
import workers
import sqlalchemy as db

# Webscrapping

I'm using tudogostoso's webpage to do the data extraction.

In [2]:
url = 'https://www.tudogostoso.com.br/receitas'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)

## Get href from all recipes

First, I created a .py file named "workers" with the functions that I'm going to use to get all href's and then I used multiprocessing to do it faster, creating a list of all href that I need.

In [ ]:
#Last page with recipes of the website

last_page = soup.select('.pagination .row a:last-of-type')[0]['href']
last_page = int(re.findall('page=\d*', last_page)[0].split('=')[-1])

In [4]:
if __name__ ==  '__main__': 
    recipes_href = []
    num_processors = 4
    p=Pool(processes = num_processors)
    output = p.map(workers.href,[i for i in range(last_page)])
    
for href in output:
    recipes_href += href

After the list is created, I'm sending it to a csv file

In [ ]:
href_df = pd.to_csv('./data/href_df.csv', index=False)

Read the href csv file

In [9]:
href_df = pd.read_csv('./data/href_df.csv')

## Send the href dataframe to the database

Creating the connection

In [11]:
engine = db.create_engine(f'{DATABASE_URL}')

I am "breaking" the dataframe into 4000-row dataframes to send to it the database faster

In [ ]:
qty_lines = href_df.shape[0]

while qty_lines > 0:
    href_first_4000 = href_df.iloc[:4000,:]
    conn = engine.connect()
    href_first_4000.to_sql('href_df', con=conn, if_exists='append', index = False)
    conn.close()
    href_df = href_df.drop(index=list(href_first_4000.index))
    qty_lines = href_df.shape[0]

## Get recipe name, preparation time, how many portions, href

Once again I'm using the workers.py file that has the create_recipe function and I'm using multiprocessing package to run faster, since there're so many recipes.

In [ ]:
if __name__ ==  '__main__': 
    num_processors = 4
    p=Pool(processes = num_processors)
    output = p.map(workers.create_recipe,[elem for elem in href_df['href']])

Send database to a csv file

In [3]:
engine = db.create_engine(f'{DATABASE_URL}')
conn = engine.connect()
recipes_df = pd.read_sql('SELECT /*+ MAX_EXECUTION_TIME(1000000) */ * from recipes.recipes_df', con=conn)

In [ ]:
recipes_df.to_json()

In [7]:
recipes_df.to_csv('./data/recipes_df.csv', index = False)

## Get recipe's images

Using the workers.py file that has the get_images function to get all url's images from the recipes

In [ ]:
if __name__ ==  '__main__':
    p = Pool(processes = 4)
    output = p.map(workers.get_images,[elem for elem in href_df['href']])